In [ ]:
import numpy as np
import pandas as pd
import json
import csv
import random
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import regularizers
import pprint
import tensorflow as tf
from tensorflow.python.framework import ops
from sklearn.model_selection import train_test_split
from sklearn import preprocessing

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
data = pd.read_csv('/content/drive/MyDrive/news.csv') # Replace with the actual path if different
data.head()


,Unnamed: 0,title,text,label
0,8476,You Can Smell Hillary’s Fear,"Daniel Greenfield, a Shillman Journalism Fello...",FAKE
1,10294,Watch The Exact Moment Paul Ryan Committed Pol...,Google Pinterest Digg Linkedin Reddit Stumbleu...,FAKE
2,3608,Kerry to go to Paris in gesture of sympathy,U.S. Secretary of State John F. Kerry said Mon...,REAL
3,10142,Bernie supporters on Twitter erupt in anger ag...,"— Kaydee King (@KaydeeKing) November 9, 2016 T...",FAKE
4,875,The Battle of New York: Why This Primary Matters,It's primary day in New York and front-runners...,REAL


In [ ]:
data

,Unnamed: 0,title,text,label
0,8476,You Can Smell Hillary’s Fear,"Daniel Greenfield, a Shillman Journalism Fello...",FAKE
1,10294,Watch The Exact Moment Paul Ryan Committed Pol...,Google Pinterest Digg Linkedin Reddit Stumbleu...,FAKE
2,3608,Kerry to go to Paris in gesture of sympathy,U.S. Secretary of State John F. Kerry said Mon...,REAL
3,10142,Bernie supporters on Twitter erupt in anger ag...,"— Kaydee King (@KaydeeKing) November 9, 2016 T...",FAKE
4,875,The Battle of New York: Why This Primary Matters,It's primary day in New York and front-runners...,REAL
...,...,...,...,...
6330,4490,State Department says it can't find emails fro...,The State Department told the Republican Natio...,REAL
6331,8062,The ‘P’ in PBS Should Stand for ‘Plutocratic’ ...,The ‘P’ in PBS Should Stand for ‘Plutocratic’ ...,FAKE
6332,8622,Anti-Trump Protesters Are Tools of the Oligarc...,Anti-Trump Protesters Are Tools of the Oligar...,FAKE
6333,4021,"In Ethiopia, Obama seeks progress on peace, se...","ADDIS ABABA, Ethiopia —President Obama convene...",REAL


In [ ]:
data = data.drop(["Unnamed: 0"], axis=1)
data

,title,text,label
0,You Can Smell Hillary’s Fear,"Daniel Greenfield, a Shillman Journalism Fello...",FAKE
1,Watch The Exact Moment Paul Ryan Committed Pol...,Google Pinterest Digg Linkedin Reddit Stumbleu...,FAKE
2,Kerry to go to Paris in gesture of sympathy,U.S. Secretary of State John F. Kerry said Mon...,REAL
3,Bernie supporters on Twitter erupt in anger ag...,"— Kaydee King (@KaydeeKing) November 9, 2016 T...",FAKE
4,The Battle of New York: Why This Primary Matters,It's primary day in New York and front-runners...,REAL
...,...,...,...
6330,State Department says it can't find emails fro...,The State Department told the Republican Natio...,REAL
6331,The ‘P’ in PBS Should Stand for ‘Plutocratic’ ...,The ‘P’ in PBS Should Stand for ‘Plutocratic’ ...,FAKE
6332,Anti-Trump Protesters Are Tools of the Oligarc...,Anti-Trump Protesters Are Tools of the Oligar...,FAKE
6333,"In Ethiopia, Obama seeks progress on peace, se...","ADDIS ABABA, Ethiopia —President Obama convene...",REAL


In [ ]:
# encoding the labels
le = preprocessing.LabelEncoder()
le.fit(data['label'])
data['label'] = le.transform(data['label'])


In [ ]:
embedding_dim = 50
max_length = 54
trunc_type = 'post'
padding_type = 'post'
oov_tok = "<OOV>"
training_size = 3000
test_portion = .1


In [ ]:
title = []
text = []
labels = []
for x in range(training_size):
    title.append(data['title'][x])
    text.append(data['text'][x])
    labels.append(data['label'][x])


In [ ]:
tokenizer1 = Tokenizer()
tokenizer1.fit_on_texts(title)
word_index1 = tokenizer1.word_index
vocab_size1 = len(word_index1) + 1

# Convert text to sequences
sequences1 = tokenizer1.texts_to_sequences(title)
padded1 = pad_sequences(sequences1, padding=padding_type, truncating=trunc_type)

# Verify shape
print(f"Padded shape: {padded1.shape}")

# Splitting data
split = int(test_portion * training_size)
training_sequences1 = padded1[split:training_size]
test_sequences1 = padded1[0:split]
test_labels = labels[0:split]
training_labels = labels[split:training_size]

Padded shape: (3000, 54)


In [ ]:
!wget http://nlp.stanford.edu/data/glove.6B.zip # Download GloVe embeddings
!unzip glove.6B.zip                            # Unzip the downloaded file

embeddings_index = {}
with open('glove.6B.50d.txt') as f:            # Now open the file
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs

# Generating embeddings
embeddings_matrix = np.zeros((vocab_size1+1, embedding_dim))
for word, i in word_index1.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embeddings_matrix[i] = embedding_vector




In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size1+1, embedding_dim, input_length=max_length, weights=[embeddings_matrix], trainable=False),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Conv1D(64, 5, activation='relu'),
    tf.keras.layers.MaxPooling1D(pool_size=4),
    tf.keras.layers.LSTM(64),
    tf.keras.layers.Dense(1, activation='sigmoid')
])
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
model.build(input_shape=(None, max_length))
model.summary()


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_1 (Embedding)              │ (None, 54, 50)              │         377,650 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 54, 50)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv1d_1 (Conv1D)                    │ (None, 50, 64)              │          16,064 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling1d_1 (MaxPooling1D)       │ (None, 12, 64)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_1 (LSTM)                        │ (None, 64)                  │          33,024 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 1)                   │              65 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 426,803 (1.63 MB)

 Trainable params: 49,153 (192.00 KB)

 Non-trainable params: 377,650 (1.44 MB)

In [ ]:
num_epochs = 10

training_padded = np.array(training_sequences1)
training_labels = np.array(training_labels)
testing_padded = np.array(test_sequences1)
testing_labels = np.array(test_labels)

history = model.fit(training_padded, training_labels, epochs=num_epochs, validation_data=(testing_padded, testing_labels), verbose=2)

print("Training Complete")

Epoch 1/10
85/85 - 3s - 33ms/step - accuracy: 0.7089 - loss: 0.5664 - val_accuracy: 0.6900 - val_loss: 0.5655
Epoch 2/10
85/85 - 1s - 16ms/step - accuracy: 0.7296 - loss: 0.5371 - val_accuracy: 0.7067 - val_loss: 0.5188
Epoch 3/10
85/85 - 1s - 17ms/step - accuracy: 0.7659 - loss: 0.4849 - val_accuracy: 0.7333 - val_loss: 0.5138
Epoch 4/10
85/85 - 1s - 15ms/step - accuracy: 0.8000 - loss: 0.4369 - val_accuracy: 0.7500 - val_loss: 0.5328
Epoch 5/10
85/85 - 1s - 17ms/step - accuracy: 0.8278 - loss: 0.3852 - val_accuracy: 0.7533 - val_loss: 0.4957
Epoch 6/10
85/85 - 2s - 29ms/step - accuracy: 0.8574 - loss: 0.3323 - val_accuracy: 0.7467 - val_loss: 0.5296
Epoch 7/10
85/85 - 3s - 39ms/step - accuracy: 0.8719 - loss: 0.2938 - val_accuracy: 0.7567 - val_loss: 0.5535
Epoch 8/10
85/85 - 3s - 33ms/step - accuracy: 0.8804 - loss: 0.2805 - val_accuracy: 0.7767 - val_loss: 0.5060
Epoch 9/10
85/85 - 2s - 22ms/step - accuracy: 0.8952 - loss: 0.2503 - val_accuracy: 0.7567 - val_loss: 0.5137
Epoch 10/1

In [ ]:
'''num_epochs = 50

training_padded = np.array(training_sequences1)
training_labels = np.array(training_labels)
testing_padded = np.array(test_sequences1)
testing_labels = np.array(test_labels)

# Calculate steps_per_epoch and validation_steps
batch_size = 32
steps_per_epoch = len(training_padded) // batch_size  # Manual calculation
validation_steps = len(testing_padded) // batch_size  # Manual calculation

# Convert NumPy arrays to TensorFlow Datasets
training_dataset = tf.data.Dataset.from_tensor_slices((training_padded, training_labels)).cache().batch(batch_size)
validation_dataset = tf.data.Dataset.from_tensor_slices((testing_padded, testing_labels)).cache().batch(batch_size)

# **The Fix:** Add .repeat() to ensure dataset cardinality is not infinite
training_dataset = training_dataset.repeat()
validation_dataset = validation_dataset.repeat()

# Use TensorFlow Datasets in model.fit() with steps_per_epoch
history = model.fit(training_dataset,
                    epochs=num_epochs,
                    validation_data=validation_dataset,
                    verbose=2,
                    steps_per_epoch=steps_per_epoch,
                    validation_steps=validation_steps)

Epoch 1/50
84/84 - 5s - 62ms/step - accuracy: 0.5982 - loss: 0.6553 - val_accuracy: 0.6771 - val_loss: 0.5986
Epoch 2/50
84/84 - 1s - 14ms/step - accuracy: 0.6934 - loss: 0.5785 - val_accuracy: 0.6944 - val_loss: 0.5316
Epoch 3/50
84/84 - 1s - 15ms/step - accuracy: 0.7256 - loss: 0.5355 - val_accuracy: 0.7014 - val_loss: 0.5137
Epoch 4/50
84/84 - 1s - 15ms/step - accuracy: 0.7594 - loss: 0.4806 - val_accuracy: 0.7083 - val_loss: 0.5143
Epoch 5/50
84/84 - 3s - 31ms/step - accuracy: 0.7946 - loss: 0.4410 - val_accuracy: 0.7361 - val_loss: 0.5111
Epoch 6/50
84/84 - 3s - 30ms/step - accuracy: 0.8280 - loss: 0.3804 - val_accuracy: 0.7743 - val_loss: 0.5582
Epoch 7/50
84/84 - 2s - 20ms/step - accuracy: 0.8197 - loss: 0.3901 - val_accuracy: 0.7257 - val_loss: 0.5239
Epoch 8/50
84/84 - 1s - 15ms/step - accuracy: 0.8527 - loss: 0.3338 - val_accuracy: 0.7396 - val_loss: 0.5984
Epoch 9/50
84/84 - 1s - 15ms/step - accuracy: 0.8733 - loss: 0.2941 - val_accuracy: 0.7326 - val_loss: 0.6277
Epoch 10/5

In [ ]:
# sample text to check if fake or not
X = "Karry to go to France in gesture of sympathy"

# detection
sequences = tokenizer1.texts_to_sequences([X])[0]
sequences = pad_sequences([sequences], maxlen=54,
                          padding=padding_type,
                          truncating=trunc_type)
if(model.predict(sequences, verbose=0)[0][0] >= 0.5):
    print("This news is True")
else:
    print("This news is false")


This news is false
